In [12]:
import os 
os.chdir('/home/idies/workspace/Temporary/Dakota_Peltzer/scratch/20220324')

In [13]:
import sys
sys.path.append('/home/idies/workspace/Storage/Dakota_Peltzer/persistent/hrpo-pipeline')

import aperturePhot as ap
from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import plateSolve
%matplotlib inline

datadir = '/home/idies/workspace/Temporary/Dakota_Peltzer/scratch/20220324/reduced'
path = Path(datadir)
os.chdir(path)

In [14]:
#Load images and plate solve them (plate solving is not strictly necessary,
#so move on without it if it doesn't work)

imname = 'PG1323-085-001R.fit'
imR = CCDData.read(imname)
check = plateSolve.autoSolve(imname,imR)
print(imname,check)
imname = 'PG1323-085-001V.fit'
imR = CCDData.read(imname)
check = plateSolve.autoSolve(imname,imR)
print(imname,check)
imname = 'PG1323-085-001B.fit'
imR = CCDData.read(imname)
check = plateSolve.autoSolve(imname,imR)
print(imname,check)

FileNotFoundError: [Errno 2] No such file or directory: '/home/idies/workspace/Storage/penny1/fall21/bin/grtrans'

In [ ]:
imR = CCDData.read('PG1323-085-001R.wcs.fit')
#imR = fits.open(path / 'Standard109954-001R_wcs.fit')

#Find stars in the image. This returns a list of stars and an image/2d array of 
#an estimate of the background.
sources,bgplane = ap.findStars(imR)

print("Found",len(sources),"stars")
#Show the image with the background subtracted and the stars found
ap.showimage(imR.data-bgplane,stars=sources,starid='label')

In [ ]:
#Generate a growth curve (i.e., measure fluxes at a range of aperture 
#sizes for all stars). This returns 4 objects: 
#rap = an array of the aperture radii
#mag = an array with estimates of the magnitude of each star
#
rap,mag,err,apers = ap.growthCurve(imR,sources)

#Plot the growth curve - use this to pick stars to use when calculating 
#the aperture correction
ap.plotGrowth(rap,mag,err,figsize=(7,7))

In [ ]:
#Use the growth curve plot to pick a star for aperture corrections. 
#This should be a list of at least one star id number. If you pick more than one
#their growth curves will be averaged (though this probably isn't the best idea).
#You want to pick a star whose growth curve gets flat and stays flat without too 
#much noise - you want the star to be reasonably bright, too. 
useForAppCorr = [1]

#You also need to pick a stopping radius - this should be the smallest radius at 
#which you think the growth curve has stopped growing. You will count the photons
#out to this radius
stoppingApertureRadius = 26.0

In [ ]:
#Plot the SNR curve for a chosen star(s) to estimate the optimum aperture. You
#should plot at least your target star and any comparison stars if you are using
#them. You can specify the stars to plot using the Nbrightest keyword as so:
#Nbrightest=X will plot the brightest X stars
#Nbrightest=[3,5,7] will plot stars with ids 3,5,7
#The function will print the optimum apertures for each of the stars, and plot 
#the SNR vs radius 
ap.plotSNR(rap,mag,err,Nbrightest=[2,4,5],figsize=(7,7))

In [ ]:
#Apply an aperture correction for a chosen optimal aperture radius. To specify
#the optimum aperture you can either specify the stars that you want to optimize
#for using optap=[list,of,one,or,more,stars], in which case it will take the 
#average of the radii that maximize SNR for the set of stars. Or you can specify 
#it directly as a floating point number. You will probably want to 
#pick the faintest stars that you will be using for this
#This function returns aperture corrected magnitudes, uncertainties, an estimate
#of the systematic error (don't use this!), and the aperture correction that was
#applied
imag,ierr,isys,apCor = ap.apertureCorrection(rap,mag,err,optap=[2,4,5],
                                            stoppingAperture=stoppingApertureRadius,
                                            stars=useForAppCorr)

imag

In [ ]:
#Show the image of the tooltip-labeled starfield (hover over stars with the mouse)
ap.showimage(imR,stars=sources,invert=True)

#Or with all stars labeled
ap.showimage(imR,stars=sources,starid='label')

#Print a table of the stars and their aperture corrected magnitudes and 
#uncertainties. These are your measurements of instrumental magnitudes
for i,m in enumerate(imag):
    print('%4d %7.3f +/- %5.3f (stat) %5.3f (sys)' % (sources['id'][i]-1,m,ierr[i],isys))

In [ ]:
#Repeat the process for another image, but with stars found in the first image
#This is useful if the automated star finder doesn't pick up all of the stars
#you need in each image, or to save you work/confusion trying to match star IDs
#multiple times.

import astropy.wcs as wcs
imV = CCDData.read('PG1323-085-001V.wcs.fit')
wcsR = wcs.WCS(imR.header)
wcsV = wcs.WCS(imV.header)
Rstarxy = np.transpose((sources['xcentroid'],sources['ycentroid']))
radec = wcsR.all_pix2world(Rstarxy,1)
Vstarxy = wcsV.all_world2pix(radec,1)

ap.showimage(imV,stars=Vstarxy,starid='label')


#Generate a growth curve (i.e., measure fluxes at a range of aperture 
#sizes for all stars)
rapV,magV,errV,apersV = ap.growthCurve(imV,Vstarxy)

#Apply an aperture correction for a chosen optimal aperture radius
imagV,ierrV,isysV,apCorrV = ap.apertureCorrection(rapV,magV,errV,optap=[2,4,5],
                                              stoppingAperture=stoppingApertureRadius,
                                              stars=useForAppCorr)

for i,m in enumerate(imagV):
    print('%4d %7.3f +/- %5.3f (stat) %5.3f (sys)' % (sources['id'][i]-1,m,ierr[i],isys))

In [ ]:
#You might need to read values from the header
imR.header['AIRMASS']